In [2]:
from itertools import product, combinations
import statistics
import numpy as np
import pandas as pd
import random

In [3]:
# FUNCTIONS TO PREPARE THE DATASET

############################################################################################

def empty_fill(df):
    taxa = ['Kingdom', 'Phylum', 'Class', 'Order', 'Family', 'Genus', 'Species']
    taxa_acronym = ['k_', 'p_', 'c_', 'o_', 'f_', 'g_', ' sp']
    # enumerate over each column in the dataframe 
    # to automatically label the empty entries according to the aformentioned labelling
    for index, taxon in enumerate(taxa):
        # only at the species level, acryonym is added at the end
        if taxon == taxa[-1]:
            df[taxon][df[taxon].isna()] = df[taxa[index-1]] + taxa_acronym[index]
        else:
            df[taxon][df[taxon].isna()] = taxa_acronym[index] + df[taxa[index-1]]     
    return df

############################################################################################

def standardize_sp(df):
    not_sub_df = df[df['Species'].str.contains(' ')]
    sub_df = df[ ~ df['Species'].str.contains(' ')]
    sub_df_copy = sub_df.copy()
    sub_df_copy['Species'] = sub_df['Genus'] + ' ' + sub_df['Species']
    return pd.concat([not_sub_df, sub_df_copy])

############################################################################################

def keep_min_max_len(df, min_max = (1000, 2000)):
    seq_len_series = df['Sequence'].apply(lambda seq: len(seq))
    min_bool = min_max[0] <= seq_len_series
    max_bool = seq_len_series <= min_max[1]
    new_df = df[min_bool & max_bool]
    return new_df

############################################################################################

def U2T(df):
    new_df = df.copy()
    new_df['Sequence'] = df['Sequence'].apply(lambda seq: seq.replace('U', 'T'))
    return new_df

############################################################################################

def min_entries(df, taxon = 'Species', min = 10):
    return df[df[taxon].map(df[taxon].value_counts()) >= min]

############################################################################################

prim = 'GGCGGACGGGTGAGTAA'
common_prim = [
    'GGCGGACGGGTGAGTAA', 'GGCGCACGGGTGCGTAA', 'GGCGAACGGGCGAGTCA', 'GGCGAAAGGGTGAGTAA',
    'GGCGACCGGGTGAGTAA', 'GGCGAACGGGTGCGTAA', 'GGCGGACGGGTGAGGAA', 'GCCAAACGGGGTAGTAA']
possible_prim = []

for indices in combinations(range(len(prim)), 14):
    for letters in product('ATGC', repeat = 3):
        letter_iter = iter(letters)
        possible_prim.append(
            ''.join([prim[x] if x in indices else letter_iter.__next__() for x in range(len(prim))]))

random.shuffle(possible_prim)
primers = common_prim + possible_prim

def find_start(seq):
    start_region = seq[:300]

    for primer in possible_prim:
        index = start_region.find(primer)
        if index > -1:
            return index

    return index

def line_up(df):
    new_df = df.copy()
    temp_df = df.copy()

    indeces = df['Sequence'].copy().apply(
        lambda seq: find_start(seq))

    temp_df['Index'] = indeces
    new_df['Sequence'] = temp_df.apply(
        lambda x: x['Sequence'][x['Index']:] if x['Index'] != -1 else np.nan, axis=1)

    return new_df.dropna()

############################################################################################

complement_dict = {
    'A': 'T', 'C': 'G', 'T': 'A', 'G': 'C', 'U':'A', 
    'Y':'R', 'R':'Y', 'W':'W', 'S':'S', 'K':'M', 'M':'K', 
    'D':'H', 'V':'B', 'H':'D', 'B':'V', 'N':'N', 'X':'X', '-':'-'}

def reverse_complement(seq):
    seq_list = list(seq)
    seq_list = [complement_dict[base] for base in seq_list]
    return ''.join(seq_list)[::-1]

def rev_comp(df):
    new_df = df.copy()
    new_df['Sequence'] = df['Sequence'].apply(lambda seq : reverse_complement(seq))
    return df

############################################################################################

def most_frequent(arr):
    # returns the most frequent element in array
    unique_nucl, counts = np.unique(arr, return_counts = True)
    return unique_nucl[counts.argmax()]

def consensus(str_lst):
    i_short = len(min(str_lst, key=len))
    # generate 2D array with nucleotides
    cons_lst = np.column_stack(([list(seq[:i_short]) for seq in str_lst]))
    return ''.join(np.apply_along_axis(most_frequent, 1, cons_lst))

def normalize(df, taxon = 'Species'):
    old_df = df.sample(frac = 1).copy()
    new_df = pd.DataFrame({
        'Kingdom':[], 'Phylum':[], 'Class':[], 'Order':[], 'Family':[], 'Genus':[], 'Species':[], 'Sequence':[]})

    # values and arrays used for distinguishing and choosing over/under-represented labels
    tax_count = old_df[taxon].value_counts()
    tax_avg = tax_count.mean()
    tax_std = statistics.stdev(tax_count)
    # over/under-representation is checked for each unique label
    unique_labels = old_df[taxon].unique()

    for label in unique_labels:
        label_count = tax_count[label]
        entries = old_df.loc[old_df[taxon] == label].copy()
        groups = round(tax_avg + tax_std + (label_count // tax_avg))

        if label_count > groups:
            new_entries = entries[:groups].copy()
            cons_seqs = []
            start = 0
            stop = label_count // groups
            for group in range(groups):
                sequences = entries[start:stop]['Sequence'].tolist()
                cons_seqs.append(consensus(sequences))
                start = stop
                stop += label_count // groups
            new_entries['Sequence'] = cons_seqs
            new_df = pd.concat([new_df, new_entries])
            
        else:
            new_df = pd.concat([new_df, entries])  
    return new_df

In [5]:
import pandas as pd
# ----------------------------------------------------------------------------------------------
# Loading the dataset as a pandas data frame ---------------------------------------------------
df = pd.read_csv('data/full_curated_dataset.csv')
print('dataset loaded', df.shape)

# Labeling empty cells with an informative value -----------------------------------------------
df0 = empty_fill(df)

# Standardizing the species columns ------------------------------------------------------------
df1 = standardize_sp(df0)

# Standardizing the sequence length and T/U ----------------------------------------------------
df2 = keep_min_max_len(df1, min_max = (1300, 1600))
df2 = U2T(df2)
print('dataset standardized', df2.shape)

# Initial restriction to reduce memory requirements --------------------------------------------
df3 = min_entries(df2, taxon = 'Species', min = 3)
print('1st restriction', df3.shape)

# Line up sequences ----------------------------------------------------------------------------
df4 = line_up(df3).drop_duplicates(subset = ['Species', 'Sequence'])
print('sequences lined up & de-duplicated', df4.shape)

dataset loaded (70785, 8)
dataset standardized (69675, 8)
1st restriction (49961, 8)
sequences lined up & de-duplicated (40561, 8)


In [10]:
# Normalize dataset by collapsing sequences of overrepresented species -------------------------
df5 = normalize(df4)
print('dataset normalized', df5.shape)

# Increase the amount of entries by adding the reverse complement sequences --------------------
rc_df = rev_comp(df5)
df6 = pd.concat([df5, rc_df], ignore_index = True)
print('total dataset with augmentation', df6.shape)

# Removal of too underrepresented species ------------------------------------------------------
df7 = min_entries(df5, taxon = 'Species', min = 5).sort_values(by = ['Family', 'Genus', 'Species'])
df7RC = min_entries(df6, taxon = 'Species', min = 5).sort_values(by = ['Family', 'Genus', 'Species'])
print('dataset species-restricted', df7.shape, df7RC.shape)

# Save train/val datasets ----------------------------------------------------------------------
df7.to_csv('df_noRC.csv', index = False)
df7RC.to_csv('df_wRC.csv', index = False)
print('dataset saved')

dataset normalized (30547, 8)
total dataset with augmentation (61094, 8)
dataset species-restricted (25446, 8) (59728, 8)
dataset saved


In [11]:
print(df7['Species'].nunique())
print(df7['Genus'].nunique())
print(df7['Family'].nunique())

1461
961
342


In [12]:
print(df7RC['Species'].nunique())
print(df7RC['Genus'].nunique())
print(df7RC['Family'].nunique())

2801
1472
417


In [13]:
df

,Kingdom,Phylum,Class,Order,Family,Genus,Species,Sequence
0,Bacteria,Acidobacteria,Acidobacteria,Acidobacteriales,Acidobacteriaceae,g_Acidobacteriaceae,g_Acidobacteriaceae sp,ACATGCAAGTCGCACGAGAAAGTGGGAGCAATCCCATGAGTACAGT...
1,Bacteria,Acidobacteria,Acidobacteria,Acidobacteriales,Acidobacteriaceae,g_Acidobacteriaceae,g_Acidobacteriaceae sp,AACGCTGGCGGCGTGCCTAACACATGCAAGTCGAACGAGAAAGTGG...
2,Bacteria,Acidobacteria,Acidobacteria,Acidobacteriales,Acidobacteriaceae,g_Acidobacteriaceae,g_Acidobacteriaceae sp,AACGCTGGCGGCGTGCCTAACACATGCAAGTCGAACGAGAAAGTGG...
3,Bacteria,Acidobacteria,Acidobacteria,Acidobacteriales,Acidobacteriaceae,g_Acidobacteriaceae,g_Acidobacteriaceae sp,AACGCTGGCGGCGTGCCTAACACATGCAAGTCGTACGAGAAAGTGG...
4,Bacteria,Acidobacteria,Acidobacteria,Acidobacteriales,Acidobacteriaceae,g_Acidobacteriaceae,g_Acidobacteriaceae sp,AACGCTGGCGGCGTGCCTAACACATGCAAGTCGAACGAGAAAGGGG...
...,...,...,...,...,...,...,...,...
70780,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Zymomonas,mobilis,AACTTGAGAGTTTGATTCTGGCTCAGAACGAACGCTGGCGGCATGC...
70781,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Zymomonas,mobilis,AACTTGAGAGTTTGATTCTGGCTCAGAACGAACGCTGGCGGCATGC...
70782,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Zymomonas,mobilis,AACTTGAGAGTTTGATTCTGGCTCAGAACGAACGCTGGCGGCATGC...
70783,Bacteria,Proteobacteria,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae,Zymomonas,mobilis,AACTTGAGAGTTTGATTCTGGCTCAGAACGAACGCTGGCGGCATGC...
